In [1]:
from ZW_model import *
from ZW_utils import std_classes,dataloading
from ZW_dataset import PSI_Dataset
import numpy as np

classes = std_classes
data_split_ratio = 0.85
batch_size = 100
max_epochs = 100
learning_rate = 1e-3
block_size = 22
n_embd = 32  # 32
n_head = 4  # 4
n_layer = 2  # 2
dropout = 0.1  # 0.1
vocab_size = len(classes)
model = PSI(vocab_size, n_embd, n_head, n_layer, block_size, dropout)

In [4]:

from config import DATA_DIRECTORY
from torch.utils.data import DataLoader
import torch
import torch.nn as nn

# layouts = np.load(DATA_DIRECTORY/"v22DF_m2_sorted_layouts.npy", allow_pickle=True)
# results = np.load(DATA_DIRECTORY/"v22DF_m2_sorted_results.npy", allow_pickle=True)
layouts = np.load("GPT_NA/initial_10k.npy", allow_pickle=True)
results = np.load("GPT_NA/results_initial_10k.npy", allow_pickle=True)
l2 = []
r2 = []
cutoff = 143.957
for i,r in enumerate(results):
    if r > 0:
        l2.append(layouts[i])
        r2.append(r)
layouts = np.asanyarray(l2)
results = np.asanyarray(r2)
from split_functions import uniqueness_check
designs,equipments = uniqueness_check(layouts)
sorted_equipments = equipments.copy()
sorted_equipments.sort()
sorted_results = []
for se in sorted_equipments:
    index = equipments.index(se)
    sorted_results.append(results[index])
eq_array = np.zeros((len(sorted_equipments),22))
for i,e in enumerate(sorted_equipments):
    for j,u in enumerate(e):
        eq_array[i,j] = u
re_array = np.array(sorted_results)
equipment_chunks = []
results_chunks = []
for equipment in sorted_equipments:
    for i in range(len(equipment)):
        candidate_chunk = equipment[:i+1]
        if candidate_chunk not in equipment_chunks:
            equipment_chunks.append(candidate_chunk)
            #checking the same chunks in eq array
            chunk_indices = np.where((eq_array[:,:i+1] == candidate_chunk).all(axis=1))[0]
            chunk_results = np.mean(re_array[chunk_indices])
            results_chunks.append(chunk_results)
#padding with 11
lengths = [len(e) for e in equipment_chunks]
max_length = max(lengths)
input_data = np.ones((len(equipment_chunks),max_length))*11
for i,e in enumerate(equipment_chunks):
    input_data[i,:len(e)] = e
input_data = torch.tensor(input_data).long()
output_data = torch.tensor(results_chunks).float()

In [9]:
max_epochs = 10
batch_size = 10
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
model = PSI(vocab_size, n_embd, n_head, n_layer, block_size, dropout)
for epoch in range(max_epochs):
    model.train()
    for i in range(0, len(input_data), batch_size):
        optimizer.zero_grad()
        batch = input_data[i:i+batch_size]
        target = output_data[i:i+batch_size]
        output = model(batch)
        print(output,target,output.shape,target.shape)


tensor([[[189.9513],
         [189.9204],
         [189.9502],
         ...,
         [189.0605],
         [189.9349],
         [189.9302]],

        [[189.9402],
         [189.9265],
         [189.8740],
         ...,
         [189.9404],
         [189.9526],
         [189.9151]],

        [[189.9493],
         [189.8972],
         [189.9387],
         ...,
         [189.9563],
         [189.9378],
         [189.8811]],

        ...,

        [[189.9256],
         [189.9427],
         [189.9388],
         ...,
         [189.9439],
         [189.9458],
         [189.9407]],

        [[189.9453],
         [189.9507],
         [189.9521],
         ...,
         [189.9500],
         [189.9505],
         [189.9243]],

        [[189.8566],
         [189.9315],
         [189.9198],
         ...,
         [189.9580],
         [189.9441],
         [189.9545]]], grad_fn=<ViewBackward0>) tensor([235.0581, 235.0581, 230.2139, 247.4888, 243.9382, 218.2243, 198.0264,
        198.0264, 198.0264, 198